# Animations

`emsarray` can produce simple animated plots.
This can be useful to visualise the change of sea surface temperature over time, for example.

In [ ]:
import datetime
import emsarray
from matplotlib.figure import Figure
from IPython.display import HTML

Set up the environment...

In [ ]:
%matplotlib notebook

# The coastlines used in the maps have some bad polygons, which causes some warnings.
# Lets turn those off.
import warnings
warnings.filterwarnings(
    "ignore", message="Shapefile shape has invalid polygon",
    category=UserWarning, module="shapefile")

`emsarray` can produce animations for any of the supported file formats.
For this example, we will start with an unstructured grid (COMPASS) file.

In [ ]:
setas = emsarray.tutorial.open_dataset("ugrid_mesh2d")
setas

In [ ]:
# Create the animation
from emsarray.utils import datetime_from_np_time

def make_title(np_time):
    """Generate a title for a frame of animation."""
    long_name = "Sea surface height"
    py_time = datetime_from_np_time(np_time)
    return f'{long_name}\nTime: {py_time:%Y-%m-%d %H:%M}'


anim = setas.ems.animate_on_figure(
    figure=Figure(figsize=(8, 5), dpi=100),
    title=make_title,
    scalar='eta',
    vector=('uav', 'vav'),
    interval=500)

HTML(anim.to_jshtml())

## Animate across a different axis

With multidimensional data, you can choose which axis to animate over.
For example, you can animate the temperature at one particular time across all depths.

For this example, let us use the Oceanmap example file.

In [ ]:
oceanmap = emsarray.tutorial.open_dataset('cfgrid_oceanmap')
oceanmap

In [ ]:
depth_anim = oceanmap.isel(Time=0).ems.animate_on_figure(
    figure=Figure(figsize=(8, 5), dpi=100),
    scalar='temp', interval=100, repeat='bounce',
    coordinate='st_ocean',
    title='Potential temperature\nDepth: {:.2f}m')

In [ ]:
HTML(depth_anim.to_jshtml())

## Creating videos

Animations can be rendered to video files in two different ways.
Both approaches require `ffmpeg` to be installed.
If you do not already have this installed, you can install it with:

```shell
$ conda install ffmpeg
```

You can render to an embeded video by calling `anim.to_html5_video()` instead.

In [ ]:
HTML(depth_anim.to_html5_video())

You can export the animation to a file using `anim.save()`.

In [ ]:
import os.path
animation_path = os.path.abspath("../oceanmap-animation.mp4")
depth_anim.save(animation_path, writer='ffmpeg_file')
print("Saved to", animation_path)